In [1]:
# Importando Librerías
import re
import json
import time
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)


In [2]:
# with open('test.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_data, file, indent=4, sort_keys=False)

In [3]:
# Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# Ingresando a página para obtener información de las partes de Jojo's
page_url = f'https://jojowiki.com/Category:JoJo%27s_Bizarre_Adventure_Parts'
driver.get(page_url)


## Hallando URL/Imagen cada parte 

In [5]:
# Encontrando partes
jojo_images = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[1]/a/img')
jojo_url = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[2]/a'
)

jojo_parts = {}
for image, url in zip(jojo_images, jojo_url):

    title = url.text
    url = url.get_attribute('href')

    image = image.get_attribute('src')

    image = image.replace('/thumb', '').split('/')
    image = '/'.join(image[:-1])

    jojo_parts[title] = {"url": url}
    jojo_parts[title]["image"] = image


# with open('data/parts/jojo_parts.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_parts, file, indent=4, sort_keys=False)

jojo_parts


{'Phantom Blood': {'url': 'https://jojowiki.com/Phantom_Blood',
  'image': 'https://static.jojowiki.com/images/b/b1/latest/20191015213555/Volume_1.jpg'},
 'Battle Tendency': {'url': 'https://jojowiki.com/Battle_Tendency',
  'image': 'https://static.jojowiki.com/images/3/34/latest/20191015213237/Volume_11.jpg'},
 'Stardust Crusaders': {'url': 'https://jojowiki.com/Stardust_Crusaders',
  'image': 'https://static.jojowiki.com/images/0/0a/latest/20191015212952/Volume_17.jpg'},
 'Diamond is Unbreakable': {'url': 'https://jojowiki.com/Diamond_is_Unbreakable',
  'image': 'https://static.jojowiki.com/images/b/ba/latest/20191015213904/Volume_36.jpg'},
 'Vento Aureo': {'url': 'https://jojowiki.com/Vento_Aureo',
  'image': 'https://static.jojowiki.com/images/4/40/latest/20191015215618/Volume_49.jpg'},
 'Stone Ocean': {'url': 'https://jojowiki.com/Stone_Ocean',
  'image': 'https://static.jojowiki.com/images/a/a6/latest/20191015214320/Volume_80.jpg'},
 'Steel Ball Run': {'url': 'https://jojowiki.co

## Hallando Info por Parte

In [6]:
jojo_data = jojo_parts.copy()


## Characters

In [7]:
characters_xpath = {'Phantom Blood': ['//*[@id="mw-content-text"]/div[2]/div[14]/div/div[2]/div/div[1]/a'],
                    'Battle Tendency': ['//*[@id="mw-content-text"]/div[2]/div[15]/div/div[2]/div/div[1]/a'],
                    'Stardust Crusaders': ['//*[@id="mw-content-text"]/div[2]/div[31]/div/div[2]/div/div[1]/p/span[1]/a',
                                           '//*[@id="mw-content-text"]/div[2]/div[31]/div/div[2]/div/div[1]/a'],
                    'Diamond is Unbreakable': ['//*[@id="mw-content-text"]/div[2]/div[22]/div/div[2]/div/div[1]/p/span[1]/a',
                                               '//*[@id="mw-content-text"]/div[2]/div[22]/div/div[2]/div/div[1]/a'],
                    'Vento Aureo': ['//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div/div[1]/p/span[1]/a',
                                    '//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div/div[1]/a'],
                    'Stone Ocean': ['//*[@id="mw-content-text"]/div[2]/div[13]/div/div[2]/div/div[1]/p/span[1]/a',
                                    '//*[@id="mw-content-text"]/div[2]/div[13]/div/div[2]/div/div[1]/a'],
                    'Steel Ball Run': ['//*[@id="mw-content-text"]/div[2]/div[18]/div/div[2]/div/div[1]/p/span[1]/a',
                                       '//*[@id="mw-content-text"]/div[2]/div[18]/div/div[2]/div/div[1]/a'],
                    'JoJolion': ['//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div/div[1]/p/span[1]/a',
                                 '//*[@id="mw-content-text"]/div[2]/div[27]/div/div[2]/div[8]/div[1]/a']}


In [8]:
for part in jojo_data.keys():
    driver.get(jojo_data[part]["url"])

    fields = driver.find_elements(
        By.XPATH, '//*[@id="mw-content-text"]/div[2]/aside/div/h3')

    datum = driver.find_elements(
        By.XPATH, '//*[@id="mw-content-text"]/div/aside/div/div')

    for field, data in zip(fields, datum):
        resume = ' '.join([driver.find_element(
            By.XPATH, f'//*[@id="mw-content-text"]/div[2]/p[{i}]').text for i in range(3, 6)])

        resume = re.sub("\[.*?\]", "", resume)
        jojo_data[part]["resume"] = resume
        jojo_data[part][field.text.lower()] = data.text

    print(part.upper(), '\t', jojo_data[part]["url"])

    xpath_elements = characters_xpath.get(part, None)

    temp = {}
    for xpath in xpath_elements:
        characters = driver.find_elements(By.XPATH, xpath)

        for character in characters:
            name = character.get_attribute('title')
            url = character.get_attribute('href')
            print(name, url)

            temp[name] = {"url": url}
    jojo_data[part]["characters"] = temp

    print('-'*45)


PHANTOM BLOOD 	 https://jojowiki.com/Phantom_Blood
Jonathan Joestar https://jojowiki.com/Jonathan_Joestar
Will Anthonio Zeppeli https://jojowiki.com/Will_Anthonio_Zeppeli
Robert E. O. Speedwagon https://jojowiki.com/Robert_E._O._Speedwagon
Dio Brando https://jojowiki.com/Dio_Brando
Erina Pendleton https://jojowiki.com/Erina_Pendleton
George Joestar I https://jojowiki.com/George_Joestar_I
Poco https://jojowiki.com/Poco
Poco's Sister https://jojowiki.com/Poco%27s_Sister
Tonpetty https://jojowiki.com/Tonpetty
Dire https://jojowiki.com/Dire
Straizo https://jojowiki.com/Straizo
Bruford https://jojowiki.com/Bruford
Tarkus https://jojowiki.com/Tarkus
Wang Chan https://jojowiki.com/Wang_Chan
Jack the Ripper https://jojowiki.com/Jack_the_Ripper
Adams https://jojowiki.com/Adams
Doobie https://jojowiki.com/Doobie
Page, Jones, Plant, and Bornnam https://jojowiki.com/Page,_Jones,_Plant,_and_Bornnam
Dario Brando https://jojowiki.com/Dario_Brando
Danny https://jojowiki.com/Danny
Tattoo https://jojowi

In [9]:
jojo_data

{'Phantom Blood': {'url': 'https://jojowiki.com/Phantom_Blood',
  'image': 'https://static.jojowiki.com/images/b/b1/latest/20191015213555/Volume_1.jpg',
  'resume': "Phantom Blood (ファントムブラッド Fantomu Buraddo) is the first part of JoJo's Bizarre Adventure. It was originally serialized in Weekly Shonen Jump from December 1986 to October 1987 under the title of JoJo's Bizarre Adventure Part 1 Jonathan Joestar: His Youth, and was collected into five volumes by Jump Comics. The title Phantom Blood was first officially used in the book JOJO A-GO!GO!, and then later used for the Bunkoban release of the manga. The part has received two anime adaptations: a 2007 film by Studio A.P.P.P. and a 2012 TV series by David Production. It was also adapted into a 2006 PlayStation 2 game by Anchor Inc. and Bandai. The story follows Jonathan Joestar as he matures with and eventually battles his adoptive brother, the cunning and merciless Dio Brando.",
  'japanese name': 'ジョジョの奇妙な冒険\nPart1 ファントムブラッド',
  'rom

## Get HTML Source

In [24]:
from pathlib import Path

for part in jojo_parts.keys():
    print(part.upper())

    path = f"data\html\{part}"
    Path(path).mkdir(parents=True, exist_ok=True)

    for character in jojo_data[part]['characters'].keys():
        driver.get(jojo_data[part]['characters'][character]['url'])

        source_code = driver.page_source

        with open(f'{path}\{character}.html', 'w', encoding="utf-8") as f:
            f.write(source_code)
        f.close()

        print(part, f'{path}\{character}.html')
    
    print('-'*65)


PHANTOM BLOOD
Phantom Blood data\html\Phantom Blood\Jonathan Joestar.html
Phantom Blood data\html\Phantom Blood\Will Anthonio Zeppeli.html
Phantom Blood data\html\Phantom Blood\Robert E. O. Speedwagon.html
Phantom Blood data\html\Phantom Blood\Dio Brando.html
Phantom Blood data\html\Phantom Blood\Erina Pendleton.html
Phantom Blood data\html\Phantom Blood\George Joestar I.html
Phantom Blood data\html\Phantom Blood\Poco.html
Phantom Blood data\html\Phantom Blood\Poco's Sister.html
Phantom Blood data\html\Phantom Blood\Tonpetty.html
Phantom Blood data\html\Phantom Blood\Dire.html
Phantom Blood data\html\Phantom Blood\Straizo.html
Phantom Blood data\html\Phantom Blood\Bruford.html
Phantom Blood data\html\Phantom Blood\Tarkus.html
Phantom Blood data\html\Phantom Blood\Wang Chan.html
Phantom Blood data\html\Phantom Blood\Jack the Ripper.html
Phantom Blood data\html\Phantom Blood\Adams.html
Phantom Blood data\html\Phantom Blood\Doobie.html
Phantom Blood data\html\Phantom Blood\Page, Jones, Pl

### Characters Features

In [ ]:
driver.get(jojo_data['Phantom Blood']['characters']['Jonathan Joestar']['url'])


In [ ]:
features = driver.find_elements(By.CLASS_NAME, "pi-item pi-data pi-item-spacing pi-border-color")
features

In [ ]:
images = driver.find_elements_by_css_selector('[id^="tabber-"]')
images

In [ ]:
images[1].text

In [ ]:
for part in jojo_parts.keys():
    print('-'*75)
    print(part.upper())
    for character in list(jojo_data[part]['characters'].keys())[:2]:
        driver.get(jojo_data[part]['characters'][character]['url'])
        features = driver.find_elements(By.CLASS_NAME, 'pi-item pi-data pi-item-spacing pi-border-color')
        
